In [1]:
from fastai.vision import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np

In [14]:
use_cuda = torch.cuda.is_available(); print(use_cuda)
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True
CUDA_LAUNCH_BLOCKING=1
%matplotlib inline
%reload_ext autoreload
%autoreload 2

True


In [12]:
path = untar_data(URLs.MNIST_TINY, fname="MNIST_FA", dest = "./data/")
tfms = get_transforms(do_flip=False)
data = (ImageList.from_folder(path)
       .split_by_folder()
       .label_from_folder()
       .transform(tfms, size =32)
       .databunch()
       .normalize())

In [16]:
class DenseBlock(nn.Module):
    def __init__(self, nin, nout, kern=1, stri=1, pad=0):
        super().__init__()
        self.conv1 = StdConv(nin, nin, stri, kern, pad)
        self.conv2 = StdConv(nin, nin, stri, kern, pad)
        self.conv3 = StdConv(nin*2, nin, stri, kern, pad)
        self.conv4 = StdConv(nin*3, nin, stri, kern, pad)
        self.conv5 = StdConv(nin*4, nout, stri, kern, pad)
    
    def forward(self, x):
        o1 = self.conv1(x)
        o2 = self.conv2(o1)
        o2c = torch.cat([o2,o1], dim=1)
        o3 = self.conv3(o2c)
        o3c = torch.cat([o2c,o3], dim=1)
        o4 = self.conv4(o3c)
        o4c = torch.cat([o3c, o4], dim=1)
        o5 = self.conv5(o4c)
        return o5